In [1]:
import sys 
sys.path.append("..") 

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp 

print("Numpy version: ", np.__version__)
print("Scipy version: ", sp.__version__)

from non_Hermitian_random_matrix import non_hermitian_symmetry_list, gap_types, is_complexification
from non_Hermitian_random_matrix import DeformHamiltonian, DeformHamiltonianGenerator

from machine_learning import Cluster
from topology_table import DeformTopologicalModel
from functools import partial
import itertools

from multiprocessing import Pool
from tools import save_data, save_data_M, load_data, load_data_M

Numpy version:  1.26.3
Scipy version:  1.11.4


/home/longyang/OSC/ml_topological_classification_non_-hermitian/cases_table/../topology_table/topological_model_deform.py:15: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit()
/home/longyang/OSC/ml_topological_classification_non_-hermitian/cases_table/../topology_table/bk/topological_model.py:15: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit()


In [2]:
def sim_func_Q(Q1, Q2):
    nQ = len(Q1)
    for n in range(nQ):
        _Q = Q1[n]+Q2[n]
        for v in np.linalg.eigvalsh(_Q):
            if np.log10(np.abs(v)) < -10:
                return 0
    return 1

def get_kpoints():
    '''
    Get the k points for 0D
    '''
    return [[1]]

def fast_similarity_func(model1, model2):
    kpoints = get_kpoints()
    Q1 = model1.calculate_Q(kpoints)
    Q2 = model2.calculate_Q(kpoints)
    return sim_func_Q(Q1, Q2)

# Parallel among different cases

In [5]:
def worker(parameter, n_dim, n_sample):
    
    non_Hermitian_symmetry_class, gap_type, n_band = parameter
    n_sample_new = n_sample
    
    
#     index_gap_type = {'real line':0, 'imaginary line':1, 'point':2}
#     class_type =  non_Hermitian_topological_classification_list_0D[non_Hermitian_symmetry_class][index_gap_type[gap_type]]
    
#     if class_type == 'Z+Z':
#         n_sample_new =  np.max([n_sample, int(10*(n_band/2+1)**2)])
#     elif class_type == "2Z+2Z":
#         n_sample_new = np.max([n_sample, int(10*(n_band/4+1)**2)])
        
    
    filename = "./data_number_of_band_test/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
        symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
    
    flag = False
    try: 
        _x, _y = load_data(filename)
    except:
        print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
            sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))
    
        flag = True

    if flag:
        generator  = DeformHamiltonianGenerator(
                n=n_band, n_dim=n_dim, 
                non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
                verbose=False)
        hamiltonians = generator.generate(n_sample=n_sample_new)
        models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
        # similarity_func = topology_comparator
        # cluster = Cluster(similarity_function=similarity_func)
        # center_indices, group_number = cluster.fit(models)

        similarity_func = fast_similarity_func
        cluster = Cluster(similarity_function=similarity_func)
        center_indices, group_number = cluster.fit(models)
        
        filename = "./data_number_of_band_test/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
            symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
        save_data(center_indices, group_number, filename)
    
    #return len(group_number)

n_dim = 0
n_sample = 1000

n_bands = 8*np.array([1,2,3])
parameters = list(itertools.product(non_hermitian_symmetry_list, gap_types, n_bands))
print("Total case: ", len(parameters))

n_core = 64      
_worker = partial(worker, n_dim=n_dim, n_sample=n_sample)
with Pool(n_core) as pool:
    pool.map(_worker, parameters)

Total case:  342


Calculating A: point dim = 0 n_band=8 n_sample=1000
Calculating A: real line dim = 0 n_band=8 n_sample=1000
Calculating A: real line dim = 0 n_band=24 n_sample=1000
Calculating AIII: real line dim = 0 n_band=16 n_sample=1000
Calculating A: imaginary line dim = 0 n_band=16 n_sample=1000

Calculating AI: point dim = 0 n_band=24 n_sample=1000
Calculating AI: real line dim = 0 n_band=24 n_sample=1000
Calculating AI: point dim = 0 n_band=8 n_sample=1000
Calculating BDI: real line dim = 0 n_band=16 n_sample=1000
Calculating AIII: imaginary line dim = 0 n_band=24 n_sample=1000

Calculating BDI: imaginary line dim = 0 n_band=24 n_sample=1000
Calculating AIII: imaginary line dim = 0 n_band=8 n_sample=1000

Calculating AI: imaginary line dim = 0 n_band=16 n_sample=1000
Calculating A: point dim = 0 n_band=24 n_sample=1000




Calculating D: real line dim = 0 n_band=24 n_sample=1000
Calculating DIII: imaginary line dim = 0 n_band=24 n_sample=1000
Calculating D: imaginary line dim = 0 n_band=16 n_s

/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:
/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:
/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:
/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:
/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:


Calculating AI+: point dim = 0 n_band=16 n_sample=1000



/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:


...

/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:


Calculating DIII: point dim = 0 n_band=24 n_sample=1000

Calculating D: imaginary line dim = 0 n_band=8 n_sample=1000

Calculating AII+: real line dim = 0 n_band=8 n_sample=1000

Calculating DIII+: imaginary line dim = 0 n_band=24 n_sample=1000

Calculating CII:S++: real line dim = 0 n_band=16 n_sample=1000

Calculating A:S: imaginary line dim = 0 n_band=8 n_sample=1000

Calculating BDI: real line dim = 0 n_band=8 n_sample=1000



/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:


Calculating CII:S++: imaginary line dim = 0 n_band=8 n_sample=1000



KeyboardInterrupt: 

#  Parallel for single case

In [3]:
def worker(parameter, n_dim, n_sample, n_core):
    
    non_Hermitian_symmetry_class, gap_type, n_band = parameter
    n_sample_new = n_sample    
    
    filename = "./data_number_of_band_test/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
        symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
    
    flag = False
    try: 
        _x, _y = load_data(filename)
    except:
        print("Calculating {sym_class}: {gap_type} dim = {n_dim} n_band={n_band} n_sample={n_sample}\n".format(
            sym_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band, n_sample=n_sample))
    
        flag = True

    if flag:
        generator  = DeformHamiltonianGenerator(
                n=n_band, n_dim=n_dim, 
                non_Hermitian_symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type,
                verbose=False)
        hamiltonians = generator.generate(n_sample=n_sample_new)
        models = [DeformTopologicalModel(hamiltonian=hamiltonian, gap_type=gap_type) for hamiltonian in hamiltonians]
        # similarity_func = topology_comparator
        # cluster = Cluster(similarity_function=similarity_func)
        # center_indices, group_number = cluster.fit(models)

        similarity_func = fast_similarity_func
        cluster = Cluster(similarity_function=similarity_func, n_core=n_core)
        center_indices, group_number = cluster.fit(models)
        
        filename = "./data_number_of_band_test/data_{symmetry_class}_{gap_type}_dim_{n_dim}_band_{n_band}.h5".format(
            symmetry_class=non_Hermitian_symmetry_class, gap_type=gap_type, n_dim=n_dim, n_band=n_band)
        save_data(center_indices, group_number, filename)
    
    #return len(group_number)

n_dim = 0
n_sample = 3000

n_bands = 8*np.array([1,2,3])
parameters = list(itertools.product(non_hermitian_symmetry_list, gap_types, n_bands))
print("Total case: ", len(parameters))

n_core = 64
_worker = partial(worker, n_dim=n_dim, n_sample=n_sample, n_core=n_core)
list(map(_worker, parameters))

Total case:  342
Calculating A: real line dim = 0 n_band=8 n_sample=3000



/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:
/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:
/tmp/ipykernel_144448/2434169657.py:6: RuntimeWarning: divide by zero encountered in log10
  if np.log10(np.abs(v)) < -10:


KeyboardInterrupt: 